# Обучение моделей

In [1]:
import numpy as np
import pandas as pd
import os
import json
from PIL import Image
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
%matplotlib inline
import seaborn as sns
import random
import cv2
from pycocotools.coco import COCO
import time

import warnings
warnings.filterwarnings('ignore')

import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

from sklearn import metrics

import torchvision.models as models
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN

from torchvision.models import mobilenet_v3_large, resnet152
import torch.nn as nn
from torch.nn import functional as F

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# !cp /content/drive/MyDrive/chkpt_model1a_d_0.pth .

In [4]:
# !cp chkpt_model1a_d_0.pth /content/drive/MyDrive
# !cp chkpt_m1a_d_0.pth /content/drive/MyDrive

In [8]:
device_id = -1
device = 'cpu' if device_id == -1 else f'cuda:{device_id}'

n_epochs = 5
batch_size = 6
classes = 2
classes1 = 156
device

'cpu'

### models DETECT

In [9]:
loss1_train = []
loss1_val = []

for epoch in range(n_epochs):
    start = time.time()
    model1.train()
    for i, batch in enumerate(data_loader_train1):
        optimizer1.zero_grad()
        loss_train = 0
        imgs, targets = batch
        imgs = list(img.to(device) for img in imgs)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        loss_dict = model1(imgs, targets)
        losses = sum(loss for loss in loss_dict.values())
        loss_train += losses.item()
        losses.backward()
        optimizer1.step()
        
        if i % 200 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {losses.item()}')
    
    loss11_train = (loss_train / i)
    loss1_train.append(loss11_train)
    
    print('Эпоха train:', epoch,'Итераций:', i, 'train loss:', (loss_train / i))
    
    for i, batch in enumerate(data_loader_val1):
        optimizer1.zero_grad()
        loss_val = 0
        imgs, targets = batch
        imgs = list(img.to(device) for img in imgs)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        with torch.no_grad():
              loss_dict = model1(imgs, targets)
        losses = sum(loss for loss in loss_dict.values())
        loss_val += losses.item()
        
        if i % 200 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {losses.item()}')
    
    loss11_val = (loss_val / i)
    loss1_val.append(loss11_val)
    
#     lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer1, step_size=, gamma=0.5)
    
    end = time.time()
    print(f"Потрачено {round((end - start) / 60, 1)} минут на {epoch} эпоху")
    print('Эпоха val:', epoch, 'Итераций:', i, 'val loss:', (loss_val / i))
    
        
    torch.save({'epoch': epoch,
                    'model_state_dict': model1.state_dict(),
                    'optimizer_state_dict': optimizer1.state_dict(),
                    'lr_scheduler_state_dict': lr_scheduler.state_dict(),
                    'loss_train': loss1_train,
                    'loss_val': loss1_val,
                    }, f'./chkpt_model1_d_{epoch}.pth')
    
    torch.save(model1.state_dict(), f'./chkpt_m1_d_{epoch}.pth')
    !cp chkpt_model1_d_{epoch}.pth /content/drive/MyDrive
    !cp chkpt_m1_d_{epoch}.pth /content/drive/MyDrive

In [ ]:
# Визуализация

In [ ]:
fig, ax  = plt.subplots()
ax.plot(loss1_train, label='Train')
ax.plot(loss1_val, label='Val')
ax.set(xlabel='Epoch', ylabel='Loss')
ax.savefig(f"./loss_d.png")
ax.legend()

### models CLASSIFIC

In [ ]:
train_loss, valid_loss = [], []
train_acc, valid_acc = [], []   

y_true_t, y_true_v = [], []
y_pred_t, y_pred_v = [], []

for epoch in range(n_epochs):
    start = time.time()
    model.train()
    train_running_loss = 0.0
    train_running_correct = 0
    train_class_correct = list(0. for i in range(classes1))
    train_class_total = list(0. for i in range(classes1))
    for i, batch in enumerate(data_loader_train22):
        optimizer.zero_grad()
        image, targets = batch['images'].to(device), batch['targets'].to(device)
#         image = list(img.to(device) for img in image)
#         targets = [t.to(device) for t in targets]
        # Forward pass.
        outputs = model(image)
        # Calculate the loss.
        loss = loss_func(outputs, targets)
        train_running_loss += loss.item()
        # Calculate the accuracy.
        _, preds = torch.max(outputs, 1)
        train_running_correct += (preds == targets).sum().item()
        correct  = (preds == targets).squeeze()
        for a in range(len(preds)):
                label = targets[a]
                # print(label)
                train_class_correct[label] += correct[a].item()
                train_class_total[label] += 1
        # Backpropagation.
        loss.backward()
        # Update the weights.
        optimizer.step()     
        
        y_true_t.extend([int(item) for item in targets])
        y_pred_t.extend([int(item) for item in preds])
        
        if i % 400 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')
    
    # Loss and accuracy for the complete epoch.
    train_epoch_loss = train_running_loss / i
    train_epoch_acc = 100. * (train_running_correct / len(df222))
    
    
    
    train_epoch_accuracy = metrics.accuracy_score(y_true_t, y_pred_t)
    train_epoch_f1_micro = metrics.f1_score(y_true_t, y_pred_t, average="micro")
    train_epoch_f1_macro =  metrics.f1_score(y_true_t, y_pred_t, average="macro")
    train_epoch_f1_weighted = metrics.f1_score(y_true_t, y_pred_t, average="weighted")
    
    
    
    print('Эпоха train:', epoch,'Итераций:', i, 'train loss:', train_epoch_loss, 'train acc:', train_epoch_acc)
    
    model.eval()
    valid_running_loss = 0.0
    valid_running_correct = 0
    # We need two lists to keep track of class-wise accuracy.
    class_correct = list(0. for i in range(classes1))
    class_total = list(0. for i in range(classes1))
    with torch.no_grad():
        for i, batch in enumerate(data_loader_val22):
#             image, targets = batch
            image, targets = batch['images'].to(device), batch['targets'].to(device)
#             image = list(img.to(device) for img in image)
#             targets = [t.to(device) for t in targets]
            # Forward pass.
            outputs = model(image)
            # Calculate the loss.
            loss = loss_func(outputs, targets)
            valid_running_loss += loss.item()
            # Calculate the accuracy.
            _, preds = torch.max(outputs, 1)
            valid_running_correct += (preds == targets).sum().item()
            # Calculate the accuracy for each class.
            correct  = (preds == targets).squeeze()
            for j in range(len(preds)):
                label = targets[j]
                # print(label)
                class_correct[label] += correct[j].item()
                class_total[label] += 1
        
            if i % 400 == 0:
                 print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')
                    
            y_true_v.extend([int(item) for item in targets])
            y_pred_v.extend([int(item) for item in preds])
        
    # Loss and accuracy for the complete epoch.
    valid_epoch_loss = valid_running_loss / i
    valid_epoch_acc = 100. * (valid_running_correct / len(df122))
    
    valid_epoch_accuracy = metrics.accuracy_score(y_true_v, y_pred_v)
    valid_epoch_f1_micro = metrics.f1_score(y_true_v, y_pred_v, average="micro")
    valid_epoch_f1_macro =  metrics.f1_score(y_true_v, y_pred_v, average="macro")
    valid_epoch_f1_weighted = metrics.f1_score(y_true_v, y_pred_v, average="weighted")
    
    end = time.time()
    print(f"Потрачено {round((end - start) / 60, 1)} минут на {epoch} эпоху")
    print('Эпоха val:', epoch, 'Итераций:', i, 'val loss:', valid_epoch_loss, 'val acc:', valid_epoch_acc)
    
    train_loss.append(train_epoch_loss)
    valid_loss.append(valid_epoch_loss)
    train_acc.append(train_epoch_acc)
    valid_acc.append(valid_epoch_acc)
    
    torch.save({'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'accuracy_train': train_epoch_accuracy,
                    'f1_micro_trainl': train_epoch_f1_micro,
                    'f1_macro_train': train_epoch_f1_macro,
                    'f1_weighted_train': train_epoch_f1_weighted,
                    'loss_train': train_loss,
                    'acc_train': train_acc,
                    'accuracy_val': valid_epoch_accuracy,
                    'f1_micro_val': valid_epoch_f1_micro,
                    'f1_macro_val': valid_epoch_f1_macro,
                    'f1_weighted_val': valid_epoch_f1_weighted,
                    'loss_val': valid_loss,
                    'acc_val': valid_acc
                    }, f'./chkpt_model_clf_{epoch}.pth')
    
    torch.save(model.state_dict(), f'./chkpt_m_clf_{epoch}.pth')
    !cp chkpt_model_clf_{epoch}.pth /content/drive/MyDrive
    !cp chkpt_m_clf_{epoch}.pth /content/drive/MyDrive

In [ ]:
# Визуализация

In [ ]:
fig, ax  = plt.subplots()
ax.plot(train_loss, label='Train')
ax.plot(valid_loss, label='Val')
ax.set(xlabel='Epoch', ylabel='Loss')
ax.savefig(f"./loss_clf.png")
ax.legend()

In [ ]:
fig, ax  = plt.subplots()
ax.plot(train_acc, label='Train')
ax.plot(valid_lacc, label='Val')
ax.set(xlabel='Epoch', ylabel='Loss')
ax.savefig(f"./loss1.png")
ax.legend()